In [ ]:
# agent behaviors for buying, selling and holding based on the probability of trading p
def buy_sell_hold(p, amount_times):
    assert p <= 0.5, "p should be smaller than 0.5"
    psis = np.zeros(amount_times)
    dice_rolls = np.random.uniform(0, 1, amount_times)
    indices = np.nonzero(dice_rolls <= 2*p)

    for index in indices[0]:
        if np.random.randint(0,2) == 0:
            psis[index] = 1
        else:
            psis[index] = -1
    return psis

# model w noise

class Model:
    def __init__(self, p: float, n: int, size: int, k: int, omega: int, b: int) -> None:
        self.n = n
        self.p = p
        self.daily_return = 1
        self.trading_volume = 0
        self.k = k
        self.omega = omega
        self.daily_returns = []
        self.count = 0
        self.b = b

        self.daily_trading_volumes = []

        
    def distribute_opinion_groups(self) -> int:
        if self.b == 0:
            # number of opinion groups
            c = (self.n / abs(self.daily_return)) ** self.omega
                

        else:
            mean = (self.n / abs(self.daily_return)) ** self.omega

            
            c = np.random.normal(loc = mean, scale = math.sqrt(mean * self.b))
            c = int(np.round(c))
            # because logically there can only be a minimum of 1 opinion group, i.e. all agents have the same opinion, we set the minimum c to be 1.
            if c <= 0:
                c = 1
            if abs(self.daily_return) >= self.n:
                c = 1
        # rounding, as we can only have whole number of opinion groups
        c = int(np.round(c))

        # number of opnion groups can maximally be the number of agents
        c = np.min([c, self.n])
            
    
        return c

    def step(self):
        self.count += 1
        c = self.distribute_opinion_groups()
        psis = buy_sell_hold(self.p, c)
        average_agents_per_group = int(np.round(self.n/c))
        return_matrix = psis * average_agents_per_group
        trading_volume = np.sum(np.abs(return_matrix))
        self.daily_return = np.sum(return_matrix) #* k
         # following the boundry conditions on returns from Feng et al. 2012 appendix 5:
        minimum_return = self.n**((self.omega-1)/self.omega)
        if np.abs(self.daily_return) < minimum_return:
            sign = -1 if self.daily_return < 0 else 1
            self.daily_return = sign*minimum_return
        maximum_return = self.n
        if np.abs(self.daily_return) > maximum_return:
            self.daily_return = np.sign(self.daily_return)*maximum_return
        self.daily_returns.append(self.daily_return)
        self.daily_trading_volumes.append(trading_volume)
        return self.count

In [ ]:
def run_ABM(p, n, size, k, omega, b, t, runs):
    ''' Runs the ABM model for a given number of runs and time steps. Returns a list of lists of daily returns and a list of lists of trading volume.'''
    ABM_sim_results_returns = []
    ABM_sim_results_volume = []
    for run in range(runs):
        model = Model(p, n, size, k, omega, b)
        for i in range(t):
            model.step()
        ABM_sim_results_returns.append(model.daily_returns)
        ABM_sim_results_volume.append(model.trading_volume)    
    ABM_sim_results_returns_array = np.array(ABM_sim_results_returns)
    ABM_sim_results_volume_array = np.array(ABM_sim_results_volume)
    return ABM_sim_results_returns, ABM_sim_results_volume